In [1]:
import requests
from lxml import html
import re
import urllib
import pandas_datareader as pdr
import numpy as np
import pandas as pd
from pandas import DataFrame,Series
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
from pyecharts import Map

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "D:\anaconda\envs\tensorflow_gpu\lib\site-packages\lml\utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [32]:
name_tp=('山东科技大学二〇一五年录取情况统计表.xls',
           '山东科技大学二〇一四年录取情况统计表.xls',
           '山东科技大学二〇一三年录取情况统计表.xls')
num_in=(30,28,23)
num_out=(33,31,26)

In [15]:
def data_deal(name,num_in,num_out):
    #省内处理
    exc=pd.read_excel(name)
    exc['Unnamed: 3'][0]='科类二'
    excin = exc[0:num_in]
    excin.columns=excin.iloc[0]
    excin=excin.drop([0,])
    excin=excin[['科类','科类二','计划','录取','投档最高','投档最低','录取\n分数','平均']]
    excin['科类']=excin['科类'].fillna(method='ffill')
    excin['投档最高']=excin['投档最高'].fillna(method='ffill')
    excin['投档最低']=excin['投档最低'].fillna(method='ffill')
    #使用多级index
    excin.index=[excin['科类'],excin['科类二']]
    del excin['科类'],excin['科类二']
    #省外处理
    excout = exc[num_out:]
    excout.columns=excout.iloc[0]
    excout=excout.drop([num_out])
    excout[['省码','省市','批次']]=excout[['省码','省市','批次']].fillna(method='ffill')
    excout = excout.fillna(method='ffill',axis=1)
    excout.index = [excout['省市'],excout['科类']]
    del excout['省码'],excout['科类']
    
    return excin,excout
excin,excout=data_deal('山东科技大学二〇一五年录取情况统计表.xls',30,33)

In [50]:
multisum=Series([])
for d,i,o in zip(name_tp,num_in,num_out):
    excin,excout=data_deal(d,i,o)
    ser=excout['计划'].groupby(excout['省市']).sum()
    ser['山东']=excin['录取'].sum()
    multisum=pd.concat([multisum,ser],axis=1)
multisum.fillna(0)

,0,计划,计划,计划
上海,0.0,10,10,10.0
云南,0.0,60,60,0.0
内蒙古,0.0,40,40,40.0
北京,0.0,9,9,9.0
吉林,0.0,40,40,40.0
四川,0.0,70,70,70.0
天津,0.0,30,30,30.0
宁夏,0.0,124,124,110.0
安徽,0.0,145,130,110.0
山东,0.0,7710,8585,8716.0


In [41]:
df=DataFrame(np.arange(16).reshape(4,4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [42]:
ser = Series(np.random.rand(4))

In [43]:
ser

0    0.405270
1    0.889145
2    0.318394
3    0.336377
dtype: float64

In [45]:
pd.concat([ser,df],axis=1)

,0,0,1,2,3
0,0.405270,0,1,2,3
1,0.889145,4,5,6,7
2,0.318394,8,9,10,11
3,0.336377,12,13,14,15
